# Text generator based on Markov Chain

First, we have to create the model that chooses the next word

This time it'll be Markov Chain

It makes predictions based on current state and probabilities of next state to come real
More info can be found here: https://en.wikipedia.org/wiki/Markov_chain

Let's get some filtered content

First, we apply a filter and copy page link

After that we start downloading html pages

In [1]:
import os

os.system('curl "https://ficbook.net/tags/1646" > ./main_text.txt')

l = ""
with open("main_text.txt", "r") as fin:
    for line in fin:
        if "jsPageJumper" in line:
            l = line
            break

#print(l)

ima = l.find("max")
#print(l[ima + 5 : ima + 5 + l[ima + 5:].find('"')])

t = int(l[ima + 5 : ima + 5 + l[ima + 5:].find('"')])
for x in range(1, min(t + 1, 100)):
    os.system('curl "https://ficbook.net/tags/1646?p={0}" > ./pages/page{0}.txt'.format(x))

                                                Страница <b><input class="jsPageJumper" type="number" min="1" max="2874" value="1"></b> из <b>2874</b>

2874


FileNotFoundError: [Errno 2] No such file or directory: './pages/page229.txt'

In [ ]:
arr = []
for i in range(1, min(100, t + 1)):
    txt = open("./pages/page" + str(i) + ".txt").read()

    i = 0

    while "readfic/" in txt[i:]:
        i += txt[i:].find("readfic/") + 8
        s = ""
        while i < len(txt) and txt[i] != '"':
            s += txt[i]
            i += 1
        arr.append(s)

#print(arr)

for num in arr:
    os.system('curl "https://ficbook.net/readfic/{0}" > ./texts/fic{0}.txt'.format(num))

with open("ficnums.txt", "a") as fout:
    for num in arr:
        print(num, file = fout)


Then we open all files we've downloaded and parse them

The result is an enormous txt file, which contains lots of text

In [5]:
with open("ficnums.txt", "r") as fin, open("fucking_huge_doc.txt", "w") as fout:
    for line in fin:
        try:
            int(line.strip())
        except:
            continue

        try:
            txt = open("./texts/fic" + line.strip() + ".txt").read()
        except:
            continue
        #txt = open("./texts/fic" + line.strip() + ".txt").read()
        #os.system("rm ./texts/fic" + line.strip() + ".txt")
        #if not "jsPartText" in txt:
            
        beg = txt.find("jsPartText")
        en = beg + txt[beg:].find("</div>")
        res = txt[beg:en + 1]

        delmode = True
        out = ""
        for c in res:
            if c == ">":
                delmode = False
            if c == "<":
                delmode = True
            if delmode:
                continue
            out += c

        out += "\n"
        out = out.replace("&quot;", '"')
        out = out.replace("&nbsp;", ' ')
        out = out.replace("&amp;", '&')
        print(out, file = fout)


Now we create our "text generator"

In [1]:
def unity(trans_prob, cur_state, q):
    st = set(trans_prob[0][cur_state[0]].keys())
    arr = list(st.unite(set(trans_prob[i][cur_state[i]])) for i in range(q))
    dic = zip(arr,[0 for _ in arr])
    for i in range(q - 1):
        for item in trans_prob[i][cur_state[i]].keys():
            dic[item]+= trans_prob[i][cur_state[i]][item] / (2 ** (i + 1))
    for item in trans_prob[q - 1][cur_state[q - 1]].keys():
            dic[item]+= trans_prob[q - 1][cur_state[q - 1]][item] / (2 ** q)
    return dic

In [2]:
import numpy as np

class MarkovChain(object):
    def __init__(self, transition_prob, splitters = [" "], s_prob = [1], ____ = 1):
        self.q = ____
        self.transition_prob = transition_prob
        #self.states = list(transition_prob.keys())
        self.splitters = splitters #["", ".", ",", ";", ":", " --"]
        self.s_prob = s_prob #[0.8, 0.05, 0.05, 0.025, 0.025, 0.05]
 
    def next_state(self, current_state, q = 1):
        arr = unity(self.transition_prob, current_state, q) #[] #list(self.transition_prob.keys())
        return np.random.choice(arr.keys(), p = list(arr.values()))
 
    def generate_states(self, current_state, no=10):
        future_states = []
        for i in range(no):
            next_state = self.next_state([current_state[t - 1] forcurrent_state], self.q)
            future_states.append(next_state + np.random.choice(self.splitters, p = self.s_prob))
            current_state = next_state
        return future_states

The next cell creates a formatted list from given text ("the text" is the name of our lot-of-content txt file and some other books)

In [3]:
txt = open("Frai_Maks__Chuchak.txt").read() + open("sorokin.txt").read() + open("fucking_huge_doc.txt").read()# + open("ViM.txt").read() + ''
text = ""

for c in txt:
    if 'а' <= c <= 'я' or 'А' <= c <= 'Я' or c == "ё" or c == "Ё":
        text += c
    #elif c in [", ", ". ", "... ", "; ", ": ", " -- ", " - ", " "]:
    #    text += " " + c
    else:
        text += " "

text = (text.lower()).split()

In [4]:
_ = [", ", ". ", "... ", "; ", ": ", " -- ", " - ", " "]
sp = dict(zip(_, [txt.count(__) for __ in _]))
sp[". "] -= 3 * sp["... "]
sp[" "] -= sp[", "] + sp[". "] + sp["... "] + sp["; "] + sp[": "] + 2 * sp[" -- "] + 2 * sp[" - "]

norm = sum(list(sp.values()))

for __ in sp:
    sp[__] /= norm

And this one counts the transition probabilities between each pair of states (it helps our model to decide, which word will be the next)

In [5]:
#some debug may be needed
#UPD: works fine enough

q = 2
trans_prob = [{} for ___ in range(q)]

for o in range(q):
    for i in range(len(text) - 1 - o):
        wd1 = text[i]
        wd2 = text[i + 1 + o]
        if wd1 not in trans_prob:
            trans_prob[o][wd1] = {}
        if wd2 not in trans_prob[o][wd1]:
            trans_prob[o][wd1][wd2] = 0
        trans_prob[o][wd1][wd2] += 1

    for wd in trans_prob[o]:
        norm = sum(list(trans_prob[o][wd].values()))
        for q in trans_prob[o][wd]:
            trans_prob[o][wd][q] /= norm

So, it's time to initialize our model

In [6]:
word_chain = MarkovChain(transition_prob = trans_prob, splitters = list(sp.keys()), s_prob = list(sp.values()), ____ = q)

The next cells are examples and tests of the model

In [10]:
toprint = word_chain.generate_states(current_state = "бля", no = np.random.randint(250, 300))

k = toprint[0]
toprint[0] = k[0].upper() + k[1:]

for i in range(len(toprint) - 1):
    if toprint[i][-2] == ".":
        k = toprint[i + 1]
        toprint[i + 1] = k[0].upper() + k[1:]

In [11]:
print(*toprint, sep = "")

Ядь он завтра, вновь себя, вернет к. Этому. Блеску кожаная перчатка. Наткнулась на самой подвеске, были аккуратные изгибистые словно стебли оплетали какой то предмет инквизитор лишь пожала плечами, и уходите; когда остынет пепел, уберете его равнодушно ответила она стала удаляться чтобы заглушить боль он завтра вновь себя вернет к этому блеску кожаная перчатка наткнулась на самой, подвеске были аккуратные. Изгибистые. Словно стебли оплетали. Какой то предмет инквизитор лишь пожала плечами и уходите когда остынет пепел уберете его равнодушно ответила она стала удаляться чтобы заглушить боль он завтра вновь себя вернет к этому блеску кожаная перчатка наткнулась на самой подвеске были аккуратные изгибистые словно стебли оплетали, какой то предмет инквизитор лишь пожала плечами и уходите когда. Остынет пепел уберете его равнодушно ответила она стала, удаляться чтобы заглушить боль он, завтра вновь себя, вернет, к. Этому блеску кожаная перчатка наткнулась на самой подвеске были аккуратные и

In [11]:
print(*word_chain.generate_states(current_state = np.random.choice(text), no = np.random.randint(20, 80)), sep = "")

косится на. крышу сносит и потом один у бьярни. сын женился на последнем сливающемся смутном, предчувствии удара скорости мало: маньяков есть консервы. ты же дома у, меня ежели уж они приползут. уже. сделала я, лучше, чем, в варшаву, все говорили и что никогда не: видал москвы, глава vi, в событиях и. действующих на дереве и входом императора, 


In [61]:
word1 = text[3785]

In [10]:
word_chain.generate_states(current_state = word1, no = 20)

['николая',
 'и',
 'ясно',
 'как',
 'бы',
 'совершенно',
 'откровенна',
 'сказала',
 'она',
 'в',
 'лесу',
 'граф',
 'вдруг',
 'при',
 'удовлетворении',
 'своих',
 'то',
 'услужливо',
 'вынул',
 'из']

In [13]:
word_chain.generate_states(current_state = np.random.choice(text), no = 40)

['ты',
 'постой',
 'пожалуста',
 'голубчик',
 'я',
 'без',
 'перевода',
 'нет',
 'андрей',
 'я',
 'здесь',
 'присядем',
 'артиллеристы',
 'сдули',
 'нагоревшие',
 'пальники',
 'офицер',
 'в',
 'редком',
 'взгляде',
 'как',
 'и',
 'та',
 'же',
 'стоявшие',
 'перед',
 'домом',
 'в',
 'нерешительности',
 'итти',
 'сударыня',
 'в',
 'долг',
 'я',
 'ложусь',
 'спать',
 'николай',
 'в',
 'русскую',
 'батарею']

Some trash code

It can be unoptimized version of some parts or some kind of attempt to first create a very fine-working thing

In [1]:
import joblib
import word2vec

/home/vsevolod/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
word2vec.word2phrase("/home/vsevolod/Desktop/Inf_project/ViM.txt", "/home/vsevolod/Desktop/Inf_project/ViM-phrases.txt", verbose = True)

Starting training using file /home/vsevolod/Desktop/Inf_project/ViM.txt
Words processed: 500K     Vocab size: 414K  
Vocab size (unigrams + bigrams): 238567
Words in train file: 584772


In [3]:
word2vec.word2vec("/home/vsevolod/Desktop/Inf_project/ViM-phrases.txt", "/home/vsevolod/Desktop/Inf_project/ViM.bin", size = 100, verbose = True)

Starting training using file /home/vsevolod/Desktop/Inf_project/ViM-phrases.txt
Vocab size: 13257
Words in train file: 462253
Alpha: 0.000371  Progress: 99.31%  Words/thread/sec: 269.67k  

In [4]:
word2vec.word2clusters("/home/vsevolod/Desktop/Inf_project/ViM.txt", "/home/vsevolod/Desktop/Inf_project/ViM-clusters.txt", 100, verbose = True)

Starting training using file /home/vsevolod/Desktop/Inf_project/ViM.txt
Vocab size: 12470
Words in train file: 490352
Alpha: 0.000102  Progress: 100.00%  Words/thread/sec: 277.98k  

In [5]:
model = word2vec.load("/home/vsevolod/Desktop/Inf_project/ViM.bin")

In [4]:
#works fine

trans_prob = {}
wd2ind = {}

i = 0
for i in range(len(text)):
    if text[i] not in wd2ind:
        wd2ind[text[i]] = []
        trans_prob[text[i]] = {}
    wd2ind[text[i]].append(i)

In [5]:
import multiprocessing as mp

In [5]:
#works fine, do not touch

for wd in trans_prob:
    arr = []
    for i in wd2ind[wd]:
        if (i < len(text) - 1):
            arr.append(text[i + 1])
    for w in set(arr):
        trans_prob[wd][w] = arr.count(w) / len(arr)

In [6]:
#does not work properly

keys = list(trans_prob.keys())

def getword(text, wd2ind, trans_prob, start, step):
    for i in range(start, len(trans_prob), step):
        wd = keys[i]
        arr = []
        for i in wd2ind[wd]:
            if (i < len(text) - 1):
                arr.append(text[i + 1])
        for w in set(arr):
            trans_prob[wd][w] = arr.count(w) / len(arr)

In [ ]:
#even this...

k = int(len(text) ** 1/2) + 1
if __name__ == "__main__":
    man = mp.Manager()
    text_ = man.list(text)
    wd2ind_ = man.dict(wd2ind)
    trans_prob_ = man.dict(trans_prob)
    for j in range(k):
        p = mp.Process(target = getword, args = (text_, wd2ind_, trans_prob_, j, k))
        p.start()
        #p.join()
trans_prob = trans_prob_